In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import librosa
import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from matplotlib import style
import pandas
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv
from sklearn import svm
import itertools
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd
from scipy.special import comb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectPercentile, GenericUnivariateSelect
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from scipy.stats import rankdata

In [2]:
DATADIR = "data"
CATEGORIES = ["Segmented_Laugh", "Segmented_NonLaugh"]

In [3]:
def createcombinations(n):
    elements = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    a = list(itertools.combinations(elements, n))
    a = np.asarray(a).astype(int)
    return a

In [4]:
def create_training_data(t_frame, t_shift, n_mfccs, component, training_data, n_features):
    count=0
    error_count = 0
    laugh_counter = 0
    nonlaugh_counter = 0
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for aud in os.listdir(path):
            if aud == '.DS_Store':
                continue
            
            aud_array , sr = librosa.load(os.path.join(path,aud), sr=None)
            count+=1

            mfccs = []
            
            try:
                mfcc = (librosa.feature.mfcc(aud_array, sr=sr,  n_mfcc=20,  win_length = int(sr*t_frame), hop_length = int(sr*t_shift))) 
                mfcc_temp = mfcc[component,:]
                #mfcc_temp = librosa.feature.delta(mfcc_temp)
                #mfcc_temp = librosa.feature.delta(mfcc_temp)
                mean_mfccs = np.mean(np.asarray(mfcc_temp),axis = 1)
                #var_mfccs = np.var(np.asarray(mfcc_temp), axis = 1)
                
                mfccs.append(mean_mfccs)
                #mfccs.append(var_mfccs)
                mfccs = np.asarray(mfccs).reshape(n_mfccs*n_features,1)
                
                training_data.append([mfccs.reshape(-1,1), class_num])
                if category == 'Segmented_Laugh':
                    laugh_counter +=1
                else:
                    nonlaugh_counter += 1
                    if nonlaugh_counter >= laugh_counter:
                        break

            except ValueError:
                pass
            
    return laugh_counter, nonlaugh_counter
    

In [5]:
def scaling(X,n_mfcc):
    for i in range(n_mfcc):
        X[:,i] = X[:,i]/(np.mean(X[:,i]))
        

    #X[:,1] = np.square(X[:,1])
        
    return X

In [6]:
n_features = 1
t_frame = 0.025
t_shift = 0.01
n_mfcc = 20
component = createcombinations(n_mfcc)

for components in component:
    training_data = []
    laugh, nonlaugh = create_training_data(t_frame, t_shift, n_mfcc, components, training_data, n_features)
    print(laugh, nonlaugh)
    
random.shuffle(training_data) 
X = []
Y = []
for features, label in training_data:
    X.append(features)
    Y.append(label)
        
X = np.array(X).reshape(-1,n_mfcc*n_features)
Y = np.array(Y)
X = scaling(X,n_mfcc)

X_train, X_test, y_train, y_test = train_test_split(
         X, Y, test_size=0.2, random_state=123)

616 604


In [7]:
#for sfs
knn = KNeighborsClassifier(n_neighbors=2) # ml_algo used = knn
sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy'
           )

pipe = Pipeline([('sfs', sfs1), 
                 ('knn', knn)])

param_grid = [
  {'sfs__k_features': [2,4,6,8,10,12,14,16],
   'sfs__estimator__n_neighbors': [1,3, 5, 10,15,17,19]}
  ] # dict. for tuning inside and outside hyperparameters i.e. no. of k_features and no. of neighbors( for knn ) 

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5,
                  iid=True,
                  refit=False)

# run gridearch
gs = gs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.9s finished

[2020-01-04 01:06:23] Features: 1/2 -- score: 0.6398114986855484[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.8s finished

[2020-01-04 01:06:24] Features: 2/2 -- score: 0.6563719028728274[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.9s finished

[2020-01-04 01:06:24] Features: 1/2 -- score: 0.643682571794345[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 o

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.8s finished

[2020-01-04 01:06:48] Features: 3/4 -- score: 0.6637350971745876[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.8s finished

[2020-01-04 01:06:49] Features: 4/4 -- score: 0.6739180140454025[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.9s finished

[2020-01-04 01:06:50] Features: 1/6 -- score: 0.6398114986855484[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.8s finished

[2020-01-04 01:06:50] Features: 2/6 -- sc

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.8s finished

[2020-01-04 01:07:10] Features: 3/6 -- score: 0.6637350971745876[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.8s finished

[2020-01-04 01:07:11] Features: 4/6 -- score: 0.6739180140454025[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.7s finished

[2020-01-04 01:07:11] Features: 5/6 -- score: 0.6931732810713702[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.7s finished

[2020-01-04 01:07:12] Features: 6/6 -- sc

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.9s finished

[2020-01-04 01:07:31] Features: 1/8 -- score: 0.6158419075616528[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.8s finished

[2020-01-04 01:07:32] Features: 2/8 -- score: 0.66708312918504[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.8s finished

[2020-01-04 01:07:32] Features: 3/8 -- score: 0.6837416299199738[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.8s finished

[2020-01-04 01:07:33] Features: 4/8 -- scor

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.8s finished

[2020-01-04 01:07:52] Features: 3/10 -- score: 0.6820217266467523[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.8s finished

[2020-01-04 01:07:52] Features: 4/10 -- score: 0.6910864377044775[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.7s finished

[2020-01-04 01:07:53] Features: 5/10 -- score: 0.7038251858408013[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.7s finished

[2020-01-04 01:07:54] Features: 6/10 -

...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished

[2020-01-04 01:33:52] Features: 11/14 -- score: 0.7409721146602182[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.5s finished

[2020-01-04 01:33:52] Features: 12/14 -- score: 0.7127417510918641[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2020-01-04 01:33:53] Features: 13/14 -- score: 0.693502815929362[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2020-01-04 01:33:53] Features: 14/1

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished

[2020-01-04 01:34:09] Features: 11/14 -- score: 0.7143965376449453[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s finished

[2020-01-04 01:34:10] Features: 12/14 -- score: 0.6939898742446514[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2020-01-04 01:34:10] Features: 13/14 -- score: 0.7272497141923894[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2020-01-04 01:34:11] Features: 14/

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished

[2020-01-04 01:34:27] Features: 11/14 -- score: 0.7148456638902498[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.5s finished

[2020-01-04 01:34:28] Features: 12/14 -- score: 0.7007186019924874[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2020-01-04 01:34:28] Features: 13/14 -- score: 0.7071451902662094[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2020-01-04 01:34:28] Features: 14/

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.6s finished

[2020-01-04 01:34:44] Features: 9/16 -- score: 0.7090599695489772[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.5s finished

[2020-01-04 01:34:45] Features: 10/16 -- score: 0.7064545104919052[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.5s finished

[2020-01-04 01:34:45] Features: 11/16 -- score: 0.6949240041514544[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.5s finished

[2020-01-04 01:34:46] Features: 12/1

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.7s finished

[2020-01-04 01:35:00] Features: 5/16 -- score: 0.6991099134411236[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.7s finished

[2020-01-04 01:35:01] Features: 6/16 -- score: 0.7208721215090642[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.7s finished

[2020-01-04 01:35:02] Features: 7/16 -- score: 0.7183161848766945[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.6s finished

[2020-01-04 01:35:02] Features: 8/16 -

In [10]:
index_0 = []
index_1 = []
result = []
for i in range(len(gs.cv_results_['params'])):
    dic = gs.cv_results_['params'][i].values()
    lis = list(dic)
    index_0.append(int(lis[0]))
    result.append(float(gs.cv_results_['mean_test_score'][i]))
    index_1.append(int(lis[1]))

    
dataframe  = pd.DataFrame(columns=['sfs__estimator__n_neighbors','sfs__k_features','Test Accuracy'])
dataframe['sfs__estimator__n_neighbors'] = np.array(index_0) 
#change the name of 0th column for different algos. i.e ex. for RF name = 'sfs__estimator__n_trees'
dataframe['sfs__k_features'] = np.array(index_1)
dataframe['Test Accuracy'] = np.array(result)

In [11]:
pipe.set_params(**gs.best_params_).fit(X_train, y_train)
print('Best features:', pipe.steps[0][1].k_feature_idx_)

print('Best score:', gs.best_score_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.2s finished

[2020-01-04 01:37:49] Features: 1/12 -- score: 0.6353184223729142[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.4s finished

[2020-01-04 01:37:51] Features: 2/12 -- score: 0.6556582347451272[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.2s finished

[2020-01-04 01:37:52] Features: 3/12 -- score: 0.6772599761554107[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

Best features: (0, 1, 2, 3, 4, 5, 6, 8, 10, 14, 16, 18)
Best score: 0.7038934426229508


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.9s finished

[2020-01-04 01:38:01] Features: 12/12 -- score: 0.7140678222121523

In [12]:
dataframe.to_csv('knn_sfs.csv')
# change the address accordingly

In [4]:
import matplotlib.inline as plt
import seaborn as sns
%matplotlib-inline
style.use('dark_background')
fig1 = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
plt.title('Sequential Backward Selection (w. StdDev)')
#plt.grid()
plt.show()

ModuleNotFoundError: No module named 'matplotlib.inline'